In [1]:
# import stuff
import os
cwd = os.getcwd()
import pandas as pd 
import numpy as np
import json

# geocoding!
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
url = 'http://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&sensor=false'


In [2]:
# geojson writing function 
# http://geoffboeing.com/2015/10/exporting-python-data-geojson/
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson


In [8]:
# load CBS or MOSF
waste_type = 'CBS'
file_name = 'NotYetConverted\\NYS_NYC_' + waste_type + '.csv'
df = pd.read_csv(file_name)
df["Full Address"] = df["Site Address"]+"New York City, NY" #+df[" ZipCode]

print('loaded "' + file_name + '" !')
print('contains ' +str(len(df.index)) + ' ' + waste_type + ' sites') 


loaded "NotYetConverted\NYS_NYC_CBS.csv" !
contains 10 CBS sites


In [9]:
# geocode using requests
for ind, row in enumerate(df["Full Address"].values):
    params = {'sensor': 'false', 'address': row}
    
    r = requests.get(url, params=params)
    results = r.json()['results']
    while not results: 
        # if you give it too many requests it will refuse your requests
        # so let's sloppily keep requesting instead
        # update this to pause between requests to avoid hitting limit
        r = requests.get(url, params=params)
        results = r.json()['results']
        
    location = results[0]['geometry']['location']
    df.loc[ind," Latitude"] = location['lat']
    df.loc[ind," Longitude"] = location['lng']
    
    # display progress
    if ind % 100 ==0: 
        print('completed ' +str(ind+1) + '/'+ str(len(df.index)) +' requests!') 
print('completed ' +str(ind+1) + '/'+ str(len(df.index)) +' requests!')   


completed 1/10 requests!
completed 10/10 requests!


In [10]:
# now write a geojson
cols = ['Full Address']
geojson = df_to_geojson(df, cols, lat=' Latitude', lon=' Longitude')
output_filename = 'Data\\' +waste_type + '_converted.json'
with open(output_filename, 'w') as output_file:
    json.dump(geojson, output_file, indent=2) 

print('saved as "'+ output_filename + '" !')


saved as "Data\CBS_converted.json" !


In [11]:
df

,Site Number,Site Type,Facility Name,County,Locality,Site Address,ZipCode,Facility Status,Expiration Date,Latitude,Longitude,Unnamed: 11,Full Address
0,2-000065,CBS,EAST RIVER GENERATING STATION,NEW YORK,NEW YORK,801 EAST 14TH STREET,10009,ACTIVE,NaN,40.728138,-73.973830,,"801 EAST 14TH STREET New York City, NY"
1,2-000240,CBS,WARDS ISLAND WWTP,NEW YORK,NEW YORK,7 WARDS ISLAND,10035,ACTIVE,NaN,40.793227,-73.921286,,"7 WARDS ISLAND New York City, NY"
2,2-000241,CBS,NORTH RIVER WWTP,NEW YORK,NEW YORK,725 WEST 135TH STREET,10031,ACTIVE,NaN,40.821329,-73.957729,,"725 WEST 135TH STREET New York City, NY"
3,2-000277,CBS,KINGSBRIDGE BUS DEPOT,NEW YORK,NEW YORK,4065 TENTH AVENUE,10034,ACTIVE,NaN,40.870688,-73.913696,,"4065 TENTH AVENUE New York City, NY"
4,2-000280,CBS,TUSKEGEE AIRMEN DEPOT,NEW YORK,NEW YORK,1552 LEXINGTON AVENUE,10029,ACTIVE,NaN,40.788032,-73.949560,,"1552 LEXINGTON AVENUE New York City, NY"
5,2-000286,CBS,MANHATTANVILLE BUS DEPOT,NEW YORK,NEW YORK,666 WEST 133RD ST.,10027,ACTIVE,NaN,40.819364,-73.957617,,"666 WEST 133RD ST. New York City, NY"
6,2-000294,CBS,MOTHER CLARA HALE BUS DEPOT,NEW YORK,NEW YORK,721 LENOX AVENUE,10039,ACTIVE,NaN,40.821592,-73.935863,,"721 LENOX AVENUE New York City, NY"
7,2-000311,CBS,MICHAEL J. QUILL DEPOT,NEW YORK,NEW YORK,525 11 AVENUE,10018,ACTIVE,NaN,40.760454,-74.000436,,"525 11 AVENUE New York City, NY"
8,2-000413,CBS,59TH ST. GENERATING STATION,NEW YORK,NEW YORK,850 TWELTH AVE.,10019,ACTIVE,NaN,40.772321,-73.993372,,"850 TWELTH AVE. New York City, NY"
9,2-000415,CBS,74TH ST. GENERATING STATION,NEW YORK,NEW YORK,506 E. 75TH ST.,10021,ACTIVE,NaN,40.767922,-73.951144,,"506 E. 75TH ST. New York City, NY"
